In [6]:
import clickhouse_connect
import pandas as pd
from IPython.core.completerlib import import_re

# if __name__ == '__main__':
#     client = clickhouse_connect.get_client(
#         host='wajbbmzq6a.europe-west4.gcp.clickhouse.cloud',
#         user='default',
#         password='3phq~SvlfTr39',
#         secure=True
#     )
#     print("Result:", client.query("SELECT 1").result_set[0][0])

In [24]:
client = clickhouse_connect.get_client(
        host='wajbbmzq6a.europe-west4.gcp.clickhouse.cloud',
        user='default',
        password='3phq~SvlfTr39',
        secure=True
    )

In [41]:
def create_database(client, database_name):
    """Создает базу данных в ClickHouse, если она еще не существует."""
    try:
        client.command(f"CREATE DATABASE IF NOT EXISTS {database_name}")
        print(f"База данных '{database_name}' успешно создана (если ее не было).")
    except Exception as e:
        print(f"Ошибка при создании базы данных '{database_name}': {e}")

def get_clickhouse_type(pandas_dtype):
    """Преобразует тип данных Pandas в соответствующий тип данных ClickHouse."""
    if pandas_dtype == 'int64':
        return 'Int64'  # Или Int32, UInt32, в зависимости от диапазона значений
    elif pandas_dtype == 'float64':
        return 'Float64'
    elif pandas_dtype == 'bool':
        return 'Bool'
    elif pandas_dtype == 'datetime64[ns]':
        return 'DateTime'
    else:
        return 'String'  # По умолчанию используем String


def get_column_types_for_clickhouse(filepath, encoding):
    """
    Определяет типы данных столбцов в CSV-файле и преобразует их в типы ClickHouse.

    Args:
        filepath (str): Путь к CSV-файлу.
        encoding (str): Кодировка файла.

    Returns:
        dict: Словарь, где ключи - имена столбцов, а значения - типы данных ClickHouse.
    """
    try:
        df = pd.read_csv(filepath, encoding=encoding)
        column_types = {}
        for column in df.columns:
            pandas_dtype = str(df[column].dtype)
            clickhouse_type = get_clickhouse_type(pandas_dtype)
            column_types[column] = clickhouse_type
        return column_types

    except Exception as e:
        print(f"Ошибка при определении типов данных: {e}")
        return None

def create_clickhouse_table(client, database, table_name, table_schema):
    """
    Создает таблицу в ClickHouse, если она не существует.

    Args:
        client: Объект клиента ClickHouse.
        database: Имя базы данных.
        table_name: Имя таблицы.
        table_schema: Строка, содержащая SQL-запрос для создания таблицы.
    """

    try:
        client.command(f"USE {database}")

        # Проверяем, существует ли таблица
        result = client.query(f"EXISTS TABLE {database}.{table_name}")
        # Проверяем, существует ли таблица
        table_exists = result.result[0][0] if hasattr(result, 'result') and result.result else False

        if not table_exists:
            client.command(table_schema)
            print(f"Таблица '{table_name}' успешно создана в базе данных '{database}'.")
        else:
            print(f"Таблица '{table_name}' уже существует в базе данных '{database}'.")

    except Exception as e:
        print(f"Ошибка при создании таблицы '{table_name}': {e}")

def insert_data_from_csv(client, database, table_name, csv_filepath, encoding='utf-16'):
    """Вставляет данные из CSV-файла в таблицу ClickHouse."""
    try:
        df = pd.read_csv(csv_filepath, encoding=encoding) #или cp1251, если utf-16 не подходит

        # Убедимся, что имена столбцов в DataFrame соответствуют именам столбцов в таблице ClickHouse.
        #При необходимости переименуйте столбцы DataFrame:
        #df.rename(columns={'старое_имя': 'новое_имя', ...}, inplace=True)

        client.insert_df(f'{database}.{table_name}', df)
        print(f"Данные из CSV-файла '{csv_filepath}' успешно вставлены в таблицу '{database}.{table_name}'.")

    except Exception as e:
        print(f"Ошибка при вставке данных из CSV-файла: {e}")

In [39]:
filepath = r'C:\Users\Misha\PycharmProjects\click_power\test\final_tabels\типы_оплаты.csv'
encoding = 'utf-16'
table_schema = 'dashminox'
table_name = 'payment_type'
escaped_column = 'ТипОплатыId'

column_types = get_column_types_for_clickhouse(filepath, encoding)

if column_types:
    print("Типы столбцов для ClickHouse:")
    for column, data_type in column_types.items():
        print(f"{column}: {data_type}")

    # Генерация SQL-запроса для создания таблицы (пример)
    create_table_statement = f"CREATE TABLE IF NOT EXISTS {table_schema}.{table_name} ("
    for column, data_type in column_types.items():
       #Обрамляем названия столбцов, содержащие пробелы или русские буквы
        escaped_column = f"`{column}`" if ' ' in column or any(ord(c) > 127 for c in column) else column
        create_table_statement += f"\n    {escaped_column} {data_type},"
    create_table_statement = create_table_statement.rstrip(",")  # Удаляем последнюю запятую
    create_table_statement += f"\n) ENGINE = MergeTree() ORDER BY {escaped_column}"
    print("\nSQL-запрос для создания таблицы:\n", create_table_statement)

Типы столбцов для ClickHouse:
Тип оплаты: String
ТипОплатыId: Int64

SQL-запрос для создания таблицы:
 CREATE TABLE IF NOT EXISTS dashminox.payment_type (
    `Тип оплаты` String,
    `ТипОплатыId` Int64
) ENGINE = MergeTree() ORDER BY `ТипОплатыId`


In [42]:
# Создаем базу данных, если ее нет
create_database(client, database_name)
# создаем таблицу
create_clickhouse_table(client, database_name, table_name, create_table_statement)
# Вставляем данные из CSV-файла
insert_data_from_csv(client, database_name, table_name, filepath, encoding)

База данных 'dashminox' успешно создана (если ее не было).
Таблица 'payment_type' успешно создана в базе данных 'dashminox'.
Данные из CSV-файла 'C:\Users\Misha\PycharmProjects\click_power\test\final_tabels\типы_оплаты.csv' успешно вставлены в таблицу 'dashminox.payment_type'.


In [38]:
import clickhouse_connect

def create_database(client, database_name):
    """Создает базу данных в ClickHouse, если она еще не существует."""
    try:
        client.command(f"CREATE DATABASE IF NOT EXISTS {database_name}")
        print(f"База данных '{database_name}' успешно создана (если ее не было).")
    except Exception as e:
        print(f"Ошибка при создании базы данных '{database_name}': {e}")


def create_clickhouse_table(client, database, table_name, table_schema):
    """
    Создает таблицу в ClickHouse, если она не существует.

    Args:
        client: Объект клиента ClickHouse.
        database: Имя базы данных.
        table_name: Имя таблицы.
        table_schema: Строка, содержащая SQL-запрос для создания таблицы.
    """

    try:
        client.command(f"USE {database}")

        # Проверяем, существует ли таблица
        result = client.query(f"EXISTS TABLE {database}.{table_name}")
        table_exists = result.result[0][0] if hasattr(result, 'result') and result.result else False

        if not table_exists:
            client.command(table_schema)
            print(f"Таблица '{table_name}' успешно создана в базе данных '{database}'.")
        else:
            print(f"Таблица '{table_name}' уже существует в базе данных '{database}'.")

    except Exception as e:
        print(f"Ошибка при создании таблицы '{table_name}': {e}")

# Пример использования:
if __name__ == '__main__':
    client = clickhouse_connect.get_client(
        host='wajbbmzq6a.europe-west4.gcp.clickhouse.cloud',
        user='default',
        password='3phq~SvlfTr39',
        secure=True
    )

    database_name = 'dashminox'
    table_name = 'payment_type' # Укажите имя таблицы

    # Создаем базу данных, если ее нет
    create_database(client, database_name)

    # Определяем схему таблицы
    table_schema = f"""
    CREATE TABLE IF NOT EXISTS {database_name}.{table_name} (
        `Тип оплаты` Nullable(String),
        `ТипОплатыId` UInt8
    ) ENGINE = MergeTree()
    ORDER BY `ТипОплатыId`
    """
    # создаем таблицу
    create_clickhouse_table(client, database_name, table_name, table_schema)

База данных 'dashminox' успешно создана (если ее не было).
Таблица 'payment_type' успешно создана в базе данных 'dashminox'.
